In [1]:
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import torch.nn.functional as F
import numpy as np
learning_rate = 1e-4
keep_prob_rate = 0.7  # dropout rate
max_epoch = 3
BATCH_SIZE = 50

DOWNLOAD_MNIST = False
if not(os.path.exists('./mnist/')) or not os.listdir('./mnist/'):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_MNIST = True


train_data = torchvision.datasets.MNIST(root='./mnist/',
                                        train=True, 
                                        transform=torchvision.transforms.ToTensor(), 
                                        download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset = train_data ,
                               batch_size= BATCH_SIZE ,
                               shuffle= True)

test_data = torchvision.datasets.MNIST(root = './mnist/',
                                       train = False)
test_x = Variable(torch.unsqueeze(test_data.test_data, dim=1),  # 添加通道维度（MNIST是单通道，将[10000,28,28]->[10000,1,28,28]）
                 volatile=True).type(torch.FloatTensor)[:500]/255.  # volatile=True表示不跟踪梯度
                                                                  # 转换为浮点张量并归一化到[0,1]
                                                                  # 取前500个测试样本
test_y = test_data.test_labels[:500].numpy()

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d( # ???
                # patch 7 * 7 ; 1  in channels ; 32 out channels ; ; stride is 1
                # padding style is same(that means the convolution opration's input and output have the same size)
                in_channels=1     , # MNIST灰度图单通道
                out_channels=32   , # 输出32个特征图
                kernel_size=7     , # 7x7卷积核
                stride=1          , # 步长1
                padding=3         , # padding=3保证输入输出尺寸相同（(28+2*3-7)/1+1=28）
            ),
            nn.ReLU(),        # activation function
            nn.MaxPool2d(2),  # pooling operation
        )
        self.conv2 = nn.Sequential( # ???
            # line 1 : convolution function, patch 5*5 , 32 in channels ;64 out channels; padding style is same; stride is 1
            # line 2 : choosing your activation funciont
            # line 3 : pooling operation function.
            nn.Conv2d(32, 64, 5, 1, 2),  # 输入32通道，输出64通道，5x5卷积核，padding2保持尺寸
            nn.ReLU(),
            nn.MaxPool2d(2),       # 再次池化，输出尺寸7x7
         

        )
        self.out1 = nn.Linear( 7*7*64 , 1024 , bias= True)   # full connection layer one

        self.dropout = nn.Dropout(keep_prob_rate)
        self.out2 = nn.Linear(1024,10,bias=True)



    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)  # flatten the output of coonv2 to (batch_size ,32 * 7 * 7)    # ???
        out1 = self.out1(x)
        out1 = F.relu(out1)
        out1 = self.dropout(out1)
        out2 = self.out2(out1)
        output = F.softmax(out2)
        return output


def test(cnn):
    global prediction
    y_pre = cnn(test_x)
    _,pre_index= torch.max(y_pre,1)
    pre_index= pre_index.view(-1)
    prediction = pre_index.data.numpy()
    correct  = np.sum(prediction == test_y)
    return correct / 500.0


def train(cnn):
    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate )
    loss_func = nn.CrossEntropyLoss()
    for epoch in range(max_epoch):
        for step, (x_, y_) in enumerate(train_loader):
            x ,y= Variable(x_),Variable(y_)
            output = cnn(x)  
            loss = loss_func(output,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if step != 0 and step % 20 ==0:
                print("=" * 10,step,"="*5,"="*5, "test accuracy is ",test(cnn) ,"=" * 10 )

if __name__ == '__main__':
    cnn = CNN()
    train(cnn)




Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:57<00:00, 173852.46it/s]


Extracting ./mnist/MNIST\raw\train-images-idx3-ubyte.gz to ./mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 101967.27it/s]


Extracting ./mnist/MNIST\raw\train-labels-idx1-ubyte.gz to ./mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



...

Extracting ./mnist/MNIST\raw\t10k-images-idx3-ubyte.gz to ./mnist/MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 1256631.19it/s]
c:\Users\86139\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\86139\AppData\Local\Temp\ipykernel_9992\3994029198.py:30: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  test_x = Variable(torch.unsqueeze(test_data.test_data, dim=1),  # 添加通道维度（MNIST是单通道，将[10000,28,28]->[10000,1,28,28]）
c:\Users\86139\anaconda3\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Extracting ./mnist/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./mnist/MNIST\raw



C:\Users\86139\AppData\Local\Temp\ipykernel_9992\3994029198.py:77: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = F.softmax(out2)


========== 20 ===== ===== test accuracy is  0.232 ==========
========== 40 ===== ===== test accuracy is  0.328 ==========
========== 60 ===== ===== test accuracy is  0.44 ==========
========== 80 ===== ===== test accuracy is  0.542 ==========
========== 100 ===== ===== test accuracy is  0.64 ==========
========== 120 ===== ===== test accuracy is  0.688 ==========
========== 140 ===== ===== test accuracy is  0.718 ==========
========== 160 ===== ===== test accuracy is  0.734 ==========
========== 180 ===== ===== test accuracy is  0.792 ==========
========== 200 ===== ===== test accuracy is  0.806 ==========
========== 220 ===== ===== test accuracy is  0.812 ==========
========== 240 ===== ===== test accuracy is  0.824 ==========
========== 260 ===== ===== test accuracy is  0.86 ==========
========== 280 ===== ===== test accuracy is  0.84 ==========
========== 300 ===== ===== test accuracy is  0.856 ==========
========== 320 ===== ===== test accuracy is  0.862 ==========
========== 340 =